# Fine-tuning BERT models on Question Type

A version of this notebook was used to fine-tune the BERT models on the training set we make available. The fine-tuned models can be found in the GitHub repository. This notebook assumes that a GPU is available. This notebook is based on the informative tutorial by Chris McCormick and Nick Ryan. (2019, July 22). BERT Fine-Tuning Tutorial with PyTorch. Retrieved from http://www.mccormickml.com

In [ ]:
# import libraries
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import random
import numpy as np



In [ ]:
# check whether GPU ia available and choose this one
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
if torch.cuda.is_available():
    torch.cuda.get_device_name(0)

In [ ]:
# make sure that the same seed is used all over the place for better reproducibility
seed_val = 30
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
tf.random.set_seed(seed_val)

# Fine-tuning the model

In [ ]:
# read the training set
df_train = pd.read_csv("rquet_trainset.csv", delimiter='\t', header=0)
df_train.sample(10)

In [ ]:
# replace the golden labels with 0,1 to make ti sutiable for BERT
df_train.replace("NISQ", 0, inplace=True)
df_train.replace("ISQ", 1, inplace=True)

In [ ]:
# load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# just run a sample tokenization to get the max_len
max_len = 0
sentences_A_train = df_train.question.values
sentences_B_train = df_train.ctx_before1.values
sentences_C_train = df_train.ctx_after1.values
for i in range(0,len(sentences_A_train), 1):
    #Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    # NOTE: depending on what exactly you would like to fine-tune, adjust the following call:
    # if you want to fine-tune only on the question itself, you only need: sentences_A_train[i]
    # if you want to fine-tune on the question and its context-before, you need: sentences_A_train[i], sentences_B_train[i]
    # if you want to fine-tune on the question and its context-after, you need: sentences_A_train[i], sentences_C_train[i]
    input_ids = tokenizer.encode(sentences_A_train[i], sentences_C_train[i], 
                                 add_special_tokens=True)
    #Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
# Run proper tokenization now
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids_train = []
attention_masks_train = []
labels_train = df_train.gold_label.values

for i in range(0,len(sentences_A_train), 1):
        # NOTE: depending on what exactly you would like to fine-tune, adjust the following call:
    # if you want to fine-tune only on the question itself, you only need: sentences_A_train[i]
    # if you want to fine-tune on the question and its context-before, you need: sentences_A_train[i], sentences_B_train[i]
    # if you want to fine-tune on the question and its context-after, you need: sentences_A_train[i], sentences_C_train[i]
    encoded_dict = tokenizer.encode_plus(
                        sentences_A_train[i], sentences_C_train[i], 
                        add_special_tokens = True,
                        truncation = True,
                        max_length = 128,          
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_train.append(encoded_dict['input_ids'])
    
    # And its attention mask.
    attention_masks_train.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)
labels_train = torch.tensor(labels_train)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences_A_train[0])
print('Token IDs:', input_ids_train[0])

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

In [ ]:
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
# Load BERT
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,  
    output_attentions = False,
    output_hidden_states = True
)

# load cuda if GPU is available
if torch.cuda.is_available():
    model.cuda()

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
# function to run the actual training/fine-tuning of the model
def run_train_and_val(epochs):
  training_stats = []

  # Measure the total training time for the whole run.
  total_t0 = time.time()


  for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
      # Perform one full pass over the training set.

      print("")
      print('\n======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('\nTraining...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0

      # Put the model into training mode. 
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):
          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)      
              print('\nBatch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          # Clear any previously calculated gradients before performing a
          # backward pass.
          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).
          outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels )
      
          loss = outputs[0]
          logits = outputs[1]
          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. 
          total_train_loss += loss.item()

          # Perform a backward pass to calculate the gradients.
          loss.backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)            
    
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("\n")
      print("\nAverage training loss: {0:.2f}".format(avg_train_loss))
      print("\nTraining epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

      print("\n")
      print("\nRunning Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()

      # Tracking variables 
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      # Evaluate data for one epoch
      for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)
        
          # Tell pytorch not to bother with constructing the compute graph during
          # the forward pass, since this is only needed for backprop (training).
          with torch.no_grad():        

              # Forward pass, calculate logit predictions.
              outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels )
            
              loss = outputs[0]
              logits = outputs[1]
            
          # Accumulate the validation loss.
          total_eval_loss += loss.item()

          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()

          # Calculate the accuracy for this batch of test sentences, and
          # accumulate it over all batches.
          total_eval_accuracy += flat_accuracy(logits, label_ids)
        

      # Report the final accuracy for this validation run.
      avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
      print("\nAccuracy: {0:.2f}".format(avg_val_accuracy))

      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)
    
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
    
      print("\nValidation Loss: {0:.2f}".format(avg_val_loss))
      print("\nValidation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
          {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
          }
      )

  print("\n")
  print("\nTraining complete!")
  print("\nTotal training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
  print("\n")
  print("\nTraining complete!")
  print("\nTotal training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



In [ ]:
from transformers import get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
epochs = 2
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
run_train_and_val(epochs)


# Evaluating the model

In [ ]:
# Run model ON TEST set
# Load the dataset into a pandas dataframe.
df_test = pd.read_csv("rquet_testset.csv", delimiter='\t', header=0)

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df_test.shape[0]))

# replace the golden labels with 0,1 to make ti sutiable for BERT
df_test.replace("NISQ", 0, inplace=True)
df_test.replace("ISQ", 1, inplace=True)

# Create sentence and label lists
sentences_A_test = df_test.question.values
sentences_B_test = df_test.ctx_before1.values
sentences_C_test = df_test.ctx_after1.values
labels_test = df_test.gold_label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_test = []
attention_masks_test = []

for i in range(0,len(sentences_A_test)):
    # NOTE: depending on what exactly you would like to evaluate, adjust the following call:
    # if you want to fine-tune only on the question itself, you only need: sentences_A_train[i]
    # if you want to fine-tune on the question and its context-before, you need: sentences_A_train[i], sentences_B_train[i]
    # if you want to fine-tune on the question and its context-after, you need: sentences_A_train[i], sentences_C_train[i]
    encoded_dict = tokenizer.encode_plus(
                        sentences_A_test[i], sentences_C_test[i],
                        add_special_tokens = True,
                        max_length = 128,         
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt', 
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_test.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_test.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)
labels_test = torch.tensor(labels_test)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids_test, attention_masks_test, labels_test)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids_test)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
total_test_accuracy = 0
nb_test_steps = 0

# Tracking variables 
predictions , true_labels, all_embeds = [], [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]
  hidden_states = outputs[1]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  i = 0
  # go through each sentence at the second from last layer:
  while i < len(hidden_states[-2]):
    # following code to get the sentence embedding from the CLS (first token of each sentence) of the 11th layer
    sentence_embedding = hidden_states[-2][i][0]
     # following code to get the sentence embedding as the average of all tokens
    # get the tokens of each sentence
    #token_vecs = hidden_states[-2][i]
    #print (token_vecs.shape)
    # average those tokens to get an average sentence embedding
    #sentence_embedding = torch.mean(token_vecs, dim=0)
    #print (sentence_embedding.shape)
    # add the embeding to the list of snetence embeddings 
    all_embeds.append(sentence_embedding)
    i += 1
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

  tmp_test_accuracy = flat_accuracy(logits, label_ids)
    
  total_test_accuracy += tmp_test_accuracy
  nb_test_steps += 1


print('    DONE.')
print("Test Accuracy: {}".format(total_test_accuracy/nb_test_steps))

In [ ]:
# Flatten the predictions and true values
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

# get ids of predictions and 
ids = df_test.ID.values

i = 0
# create new Dataframe to hold the fine-tuned embeds
df_embeds = pd.DataFrame()

# add each prediction to the new dataframe
for pred in all_embeds:
  # convert tensor to numpy array
  numpy_pred = pred.cpu().numpy()
  reshaped_numpy = np.reshape(numpy_pred, (1,768))
  numpy_df = pd.DataFrame(reshaped_numpy)
  # append
  df_embeds = df_embeds.append(numpy_df,ignore_index=True)
  #print (df_embeds)  
  i +=1

  
print (i)
# insert the ids at the front
df_embeds.insert(0,"ID",ids)
df_embeds

In [ ]:
# write dataframe to csv file
df_embeds.to_csv("fine-tuned_bert_embeds_on_queAndCtxAfter_testset.csv")

In [ ]:
# Save model if necessary
model_save_name = 'que_contextAfter_fine-tuned_bert.pt'
torch.save(model.state_dict(), model_save_name)